In [1]:
%matplotlib inline
#%matplotlib notebook
#%matplotlib widget
import matplotlib 
import numpy as np
import pandas as pd
import os, sys
#import ipywidgets
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection    
 
# use LaTeX, choose nice some looking fonts and tweak some settings
matplotlib.rc('font', family='serif')
matplotlib.rc('font', size=16)
matplotlib.rc('legend', fontsize=16)
matplotlib.rc('legend', numpoints=1)
matplotlib.rc('legend', handlelength=1.5)
matplotlib.rc('legend', frameon=False)
matplotlib.rc('xtick.major', pad=7)
matplotlib.rc('xtick.minor', pad=7)
matplotlib.rc('text', usetex=True)
# matplotlib.rc('text.latex', 
#               preamble=[r'\usepackage[T1]{fontenc}',
#                         r'\usepackage{amsmath}',
#                         r'\usepackage{txfonts}',
#                         r'\usepackage{textcomp}'])

matplotlib.rc('figure', figsize=(12, 9))

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device('cuda:0')

In [38]:
# torch.tensor.clamp 함수 = np.array.clip  함수
xx = torch.tensor([[1,2,3,4], [2,3,4,5], [3,4,5,6]], dtype=torch.float)
x2=xx.clamp(min=2.0, max=4.0)

yy = np.array(xx)
y2=yy.clip(min = 2.0, max=4.0)

(np.array(x2)==y2).all()

True

## Numpy를 통해 구현한 순전파와 역전파.

- $x$ : 입력값
- $y$ : 출력값

In [5]:
# N :배치 크기
# D_in : 입력 차원
# H : 은닉층의 차원
# D_out : 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성합니다.
# np.random.randn : 표준정규분포를 따르는 난수 발생.
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# 무작위로 가중치를 초기화합니다.
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(1000):
    # 순전파 단계: 예측값 y를 계산합니다.
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # 손실(loss)을 계산하고 출력합니다.
    loss = np.square(y_pred - y).sum()
    if t%100 == 0:
        print(t, loss)

    # 손실에 따른 w1, w2의 변화도를 계산하고 역전파합니다.
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # 가중치를 갱신합니다.
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 39783166.91245361
100 980.0078790535046
200 9.130226112970881
300 0.1282972340540697
400 0.002151284614535909
500 4.034382334978946e-05
600 8.152841189633685e-07
700 1.7316900406064005e-08
800 3.802098951774234e-10
900 8.534367441378677e-12


In [22]:
dtype = torch.float

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
# requires_grad=False로 설정하여 역전파 중에 이 Tensor들에 대한 변화도를 계산할
# 필요가 없음을 나타냅니다. (requres_grad의 기본값이 False이므로 아래 코드에는
# 이를 반영하지 않았습니다.)
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 가중치를 저장하기 위해 무작위 값을 갖는 Tensor를 생성합니다.
# requires_grad=True로 설정하여 역전파 중에 이 Tensor들에 대한
# 변화도를 계산할 필요가 있음을 나타냅니다.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(1000):
    # 순전파 단계: Tensor 연산을 사용하여 예상되는 y 값을 계산합니다. 이는 Tensor를
    # 사용한 순전파 단계와 완전히 동일하지만, 역전파 단계를 별도로 구현하지 않아도
    # 되므로 중간값들에 대한 참조(reference)를 갖고 있을 필요가 없습니다.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # Tensor 연산을 사용하여 손실을 계산하고 출력합니다.
    # loss는 (1,) 형태의 Tensor이며, loss.item()은 loss의 스칼라 값입니다.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 0:
        print(t, loss.item())

    # autograd를 사용하여 역전파 단계를 계산합니다. 이는 requires_grad=True를
    # 갖는 모든 Tensor에 대해 손실의 변화도를 계산합니다. 이후 w1.grad와 w2.grad는
    # w1과 w2 각각에 대한 손실의 변화도를 갖는 Tensor가 됩니다.
    loss.backward()

    # 경사하강법(gradient descent)을 사용하여 가중치를 수동으로 갱신합니다.
    # torch.no_grad()로 감싸는 이유는 가중치들이 requires_grad=True이지만
    # autograd에서는 이를 추적할 필요가 없기 때문입니다.
    # 다른 방법은 weight.data 및 weight.grad.data를 조작하는 방법입니다.
    # tensor.data가 tensor의 저장공간을 공유하기는 하지만, 이력을
    # 추적하지 않는다는 것을 기억하십시오.
    # 또한, 이를 위해 torch.optim.SGD 를 사용할 수도 있습니다.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듭니다.
        w1.grad.zero_()
        w2.grad.zero_()

0 35434952.0
100 422.3948974609375
200 10.741645812988281
300 7.255825996398926
400 5.640827655792236
500 9.950870513916016
600 5.928495407104492
700 8.997459411621094
800 9.814862251281738
900 5.415840148925781


In [40]:
x = torch.ones(1, requires_grad=True)
print(x)
y = x + 2
print(y)
print("===== Run backward =====")
y.backward()
print(y)


tensor([1.], requires_grad=True)
tensor([3.], grad_fn=<AddBackward0>)
===== Run backward =====
tensor([3.], grad_fn=<AddBackward0>)


In [48]:
x = torch.ones(2, 2, requires_grad=True)
print("x=", x)

y = x + 2
print("y=", y)

z = y * y * 3
out = z.mean()
print("z=", z)
print("out=", out)

print("===== out.backward ===")
out.backward()
print(x.grad)

x= tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
y= tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
z= tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
out= tensor(27., grad_fn=<MeanBackward0>)
===== out.backward ===
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


$$
\text{out}=\overline{z}=\dfrac{1}{4}\sum_{i,\,j=1}^2 z_{ij} =\dfrac{1}{4}\sum_{i,\,j=1}^2 3{y_{ij}}^2 = \dfrac{1}{4}\sum_{i,\,j=1}^2 3(x_{ij}+2)^2 \\
\dfrac{\partial }{\partial x} \text{out}= \sum \dfrac{3}{2}(x+2)
$$